In [1]:
import pandas as pd
import numpy as np
import numpy_financial as npf
from plotnine import *
from babel.numbers import format_currency, format_percent
import math
pd.set_option('mode.chained_assignment', None)

In [42]:
presto_merged_df = pd.read_csv('presto_merged.csv')
presto_merged_df['branch'] = 'presto'

if_merged_df = pd.read_csv('if_merged.csv')
if_merged_df['branch'] = 'instantfunding'

merged_df = pd.concat([presto_merged_df, if_merged_df])

In [43]:
merged_df['release_quarter'] = np.select(
    [merged_df['release_month'] <= 3,
    merged_df['release_month'] <= 6,
    merged_df['release_month'] <= 9,
    merged_df['release_month'] <= 12],
    ['Q1', 'Q2', 'Q3', 'Q4']
)

In [63]:
loans_df = merged_df.groupby(['release_year', 'release_quarter', 'release_month', 'Loan #'])['Principal Amount'].max().reset_index()

In [ ]:
disbursed_quarterly_df = loans_df.groupby(['release_year', 'release_quarter'])['Principal Amount'].sum().reset_index()

In [57]:
disbursed_quarterly_df

,release_year,release_quarter,Principal Amount
0,2022,Q1,1644158.51
1,2022,Q2,1740188.81
2,2022,Q3,1733797.25
3,2022,Q4,1916849.42
4,2023,Q1,1974819.55
5,2023,Q2,1971947.08
6,2023,Q3,1892863.51
7,2023,Q4,2142558.68
8,2024,Q1,2026934.98
9,2024,Q2,264610.83


In [58]:
disbursed_quarterly_df.to_csv('disbursed_by_quarter.csv', index=False)

In [64]:
disbursed_monthly_df = loans_df.groupby(['release_year', 'release_month'])['Principal Amount'].sum().reset_index()

In [65]:
disbursed_monthly_df

,release_year,release_month,Principal Amount
0,2022,1,600009.92
1,2022,2,525688.48
2,2022,3,518460.11
3,2022,4,540666.45
4,2022,5,629469.00
5,2022,6,570053.36
6,2022,7,457486.47
7,2022,8,659741.76
8,2022,9,616569.02
9,2022,10,577541.99


In [66]:
disbursed_monthly_df.to_csv('disbursed_by_month.csv', index=False)

In [60]:
collected_df = merged_df.groupby(['release_year', 'release_quarter', 'release_month', 'weeks_since_release'])['Total Paid Amount'].sum().reset_index()
collected_df.to_csv('collected_by_week.csv', index=False)

In [2]:
presto_df = pd.read_csv('presto_monthly_cashflows.csv')

In [4]:
presto_df['branch'] = 'presto'

In [6]:
if_df = pd.read_csv('if_monthly_cashflows.csv')

In [7]:
if_df['branch'] = 'instantfunding'

In [8]:
combined_df = pd.concat([presto_df, if_df])

In [9]:
combined_df

,release_year,release_month,collection_year,collection_month,Total Paid Amount,Principal Amount,branch
0,2022,1,2022.0,1.0,52166.55,273574.23,presto
1,2022,1,2022.0,2.0,113973.97,273574.23,presto
2,2022,1,2022.0,3.0,108295.19,273574.23,presto
3,2022,1,2022.0,4.0,78513.48,273574.23,presto
4,2022,1,2022.0,5.0,38279.40,273574.23,presto
...,...,...,...,...,...,...,...
399,2024,2,2024.0,2.0,73023.36,376913.88,instantfunding
400,2024,2,2024.0,3.0,164753.28,376913.88,instantfunding
401,2024,2,2024.0,4.0,99283.67,376913.88,instantfunding
402,2024,3,2024.0,3.0,63213.28,313426.86,instantfunding


In [35]:
combined_df['collection_year'] = combined_df['collection_year'].astype(int)

In [36]:
combined_df['release_quarter'] = np.select(
    [combined_df['release_month'] <= 3,
    combined_df['release_month'] <= 6,
    combined_df['release_month'] <= 9,
    combined_df['release_month'] <= 12],
    ['Q1', 'Q2', 'Q3', 'Q4']
)

combined_df['collection_quarter'] = np.select(
    [combined_df['collection_month'] <= 3,
    combined_df['collection_month'] <= 6,
    combined_df['collection_month'] <= 9,
    combined_df['collection_month'] <= 12],
    ['Q1', 'Q2', 'Q3', 'Q4']
)

In [40]:
combined_agg1_df = combined_df.groupby(
    ['branch', 'release_year', 'release_quarter', 'collection_year', 'collection_quarter']
)[['Total Paid Amount', 'Principal Amount']].sum().reset_index()

combined_agg1_df

,branch,release_year,release_quarter,collection_year,collection_quarter,Total Paid Amount,Principal Amount
0,instantfunding,2022,Q1,2022,Q1,513269.50,1810955.37
1,instantfunding,2022,Q1,2022,Q2,636679.22,2635459.17
2,instantfunding,2022,Q1,2022,Q3,60341.18,2635459.17
3,instantfunding,2022,Q1,2022,Q4,12584.00,2635459.17
4,instantfunding,2022,Q1,2023,Q1,11986.77,2635459.17
...,...,...,...,...,...,...,...
105,presto,2023,Q4,2024,Q1,785945.09,2893743.57
106,presto,2023,Q4,2024,Q2,62936.24,964581.19
107,presto,2024,Q1,2024,Q1,557878.82,1909078.72
108,presto,2024,Q1,2024,Q2,242677.48,931945.65


In [38]:
combined_agg1_df['% Collected'] = combined_agg1_df['Total Paid Amount'] / combined_agg1_df['Principal Amount']

In [39]:
combined_agg1_df.query('(release_year==collection_year) & (release_quarter=="Q1") & (collection_quarter=="Q1")')

,branch,release_year,release_quarter,collection_year,collection_quarter,Total Paid Amount,Principal Amount,% Collected
0,instantfunding,2022,Q1,2022,Q1,513269.50,1810955.37,0.283425
34,instantfunding,2023,Q1,2023,Q1,807233.28,2553656.33,0.316109
53,instantfunding,2024,Q1,2024,Q1,654963.78,2281200.39,0.287114
55,presto,2022,Q1,2022,Q1,482963.93,1558911.46,0.309808
89,presto,2023,Q1,2023,Q1,481906.02,1426474.95,0.337830
107,presto,2024,Q1,2024,Q1,557878.82,1909078.72,0.292224


In [20]:
combined_agg_df = combined_df.groupby(
    ['release_year', 'release_quarter', 'collection_year', 'collection_quarter']
)[['Total Paid Amount', 'Principal Amount']].sum().reset_index()

In [24]:
combined_agg_df['% Collected'] = combined_agg_df['Total Paid Amount'] / combined_agg_df['Principal Amount']

In [25]:
combined_agg_df.query('(release_year==collection_year) & (release_quarter=="Q1") & (collection_quarter=="Q1")')

,release_year,release_quarter,collection_year,collection_quarter,Total Paid Amount,Principal Amount,% Collected
0,2022,Q1,2022.0,Q1,996233.43,3369866.83,0.295630
34,2023,Q1,2023.0,Q1,1289139.30,3980131.28,0.323894
53,2024,Q1,2024.0,Q1,1212842.60,4190279.11,0.289442
